In [5]:
import numpy as np
from random import randint
import matplotlib.pyplot as plt
import random
from collections import deque 
import time
MIN_X = -100
MIN_Y = -100
MIN_Z = 0
MAX_X = 100
MAX_Y = 100
MAX_Z = 50
NUMBER_OF_CONNECTIONS = 2


class City:
    def __init__(self) -> None:
        self.cords = np.array([
            randint(MIN_X, MAX_X),
            randint(MIN_Y, MAX_Y),
            randint(MIN_Z, MAX_Z)
        ])
        self.distances = []


class WebOfCities:
    def __init__(self, number_of_cities) -> None:
        self.number_of_cities = number_of_cities
        self.cities = []
        self.create_list_of_cities()

    def create_list_of_cities(self):
        self.cities = [City() for _ in range(0, self.number_of_cities)]

    def create_weighted_connections_100(self):
        for city in self.cities:
            distance = []
            for i in range(0, self.number_of_cities):
                if city.cords[2]  > self.cities[i].cords[2]:
                    distance.append(np.linalg.norm(city.cords - self.cities[i].cords)*0.9)
                elif city.cords[2]  < self.cities[i].cords[2]:
                    distance.append(np.linalg.norm(city.cords - self.cities[i].cords)*1.1)
                else:
                    distance.append(np.linalg.norm(city.cords - self.cities[i].cords))
            city.distances = distance

    def create_weighted_connections_80(self):
        for city in self.cities:
            distance = []
            for i in range(0, self.number_of_cities):
                if city.cords[2]  > self.cities[i].cords[2]:
                    actual_distance = np.linalg.norm(city.cords - self.cities[i].cords)*0.9
                elif city.cords[2]  < self.cities[i].cords[2]:
                    actual_distance = np.linalg.norm(city.cords - self.cities[i].cords)*1.1
                else:
                    actual_distance = np.linalg.norm(city.cords - self.cities[i].cords)
                if random.random() > 0.2:
                    distance.append(actual_distance)
                else:
                    distance.append(0)
            city.distances = distance

    def create_connections_100(self):
        for city in self.cities:
            distance = [np.linalg.norm(city.cords - self.cities[i].cords) for i in range(0, len(self.cities))]
            city.distances = distance
    
    def create_connections_80(self):
        for city in self.cities:
            distance = []
            for i in range(0, self.number_of_cities):
                if random.random() > 0.2:
                    distance.append(np.linalg.norm(city.cords - self.cities[i].cords))
                else:
                    distance.append(0)
            city.distances = distance

class Graph:
    def __init__(self, cities) -> None:
        self.cities = cities

    def create_adjacency_matrix(self):
        adjacency_matrix_ls = []
        for city in self.cities:
            list_of_distance = []
            for distance in city.distances:
                list_of_distance.append(distance)
            adjacency_matrix_ls.append(list_of_distance)
        adjacency_matrix = np.array(adjacency_matrix_ls)
        return adjacency_matrix

    def create_adjacency_weighted_matrix(self):
        adjacency_matrix_ls = []
        for city in self.cities:
            list_of_distance = []
            for distance in city.distances:
                list_of_distance.append(distance)
            adjacency_matrix_ls.append(list_of_distance)
        adjacency_matrix = np.array(adjacency_matrix_ls)
        return adjacency_matrix

In [6]:
class PathCounter:
    def __init__(self, adjacency_matrix, all_paths) -> None:
        self.adjacency_matrix = adjacency_matrix
        self.all_paths = all_paths
        self.all_distances = []
        self.index_of_min_distance = 0

    def count_all_distances(self):
        for path in self.all_paths:
            distance = 0
            for i in range(len(path)-1):
                distance += self.adjacency_matrix[path[i]][path[i+1]]
                # if self.adjacency_matrix[path[i]][path[i+1]] == 0:
                #    print("error")
            self.all_distances.append(distance)

    def print_min_possible_path(self):
        min_distance = min(self.all_distances)
        self.index_of_min_distance = self.all_distances.index(min_distance)
        path = self.all_paths[self.index_of_min_distance]
        distance = min_distance + self.adjacency_matrix[path[-1]][0]
        path.append(0)
        print(f"min distance is {distance} for path")
        print(path)
    
    def count_path(self):
        distance = 0
        for i in range(len(self.all_paths)-1):
            distance += self.adjacency_matrix[self.all_paths[i]][self.all_paths[i+1]]
                # if self.adjacency_matrix[path[i]][path[i+1]] == 0:
                #    print("error")
        print(f"min distance is {distance} for path")
        print(self.all_paths)

In [7]:
import random


class AlgorythmACO:
    def __init__(self, adjacency_matrix) -> None:
        self.adjacency_matrix = adjacency_matrix
        self.visited_cities = np.full((len(adjacency_matrix)), -1)
        self.visited_cities[0] = 0
        self.feromon = np.full(len(adjacency_matrix), 0.01)
        self.cities = np.arange(0, len(adjacency_matrix))
        self.dead_end = False
        self.feromon_matrix = np.full_like(adjacency_matrix, 0.01)


    def search(self, num_of_iter, num_of_ants):
        for _ in range(num_of_iter):
            num_of_all_cities = len(self.adjacency_matrix)
            all_paths = []
            for _ in range(num_of_ants):
                visited_cities_num = 1
                path = [0]
                choosen_city = 0
                self.visited_cities = np.full((num_of_all_cities), -1)
                self.visited_cities[0] = 0
                self.dead_end = False
                while visited_cities_num < num_of_all_cities:
                    rem_cities = self.cities[self.visited_cities == -1]
                    #print(self.visited_cities)
                    #print([self.visited_cities == -1])
                    #print(self.feromon.shape)
                    #print(self.visited_cities.shape)
                    self.feromon = self.feromon_matrix[choosen_city][:]
                    rem_feromon = self.feromon[self.visited_cities == -1]
                    cur_city = choosen_city
                    choosen_city = random.choices(rem_cities, weights=rem_feromon, k=1)[0]
                    if self.adjacency_matrix[cur_city][choosen_city] != 0:
                        path.append(choosen_city)
                        self.visited_cities[choosen_city] = choosen_city
                        visited_cities_num += 1
                    else:
                        self.dead_end = True
                        break
                if not self.dead_end and self.adjacency_matrix[choosen_city][0] != 0:
                    path.append(0)
                    all_paths.append(path)
            self.feromon_matrix = self.feromon_matrix * 0.8
            self.spread_feromon(all_paths)
            #print(self.feromon_matrix)
            pass
        return all_paths
            # self.feromon = np.array(self.spread_feromon(all_paths))

    def spread_feromon(self, all_paths):
        all_distances = []
        for path in all_paths:
            distance = 0
            for i in range(len(path)-1):
                distance += self.adjacency_matrix[path[i]][path[i+1]]
            all_distances.append(distance)
        feromons_connections_val = [(len(path)*10)/distance for distance in all_distances]
        for i, path in enumerate(all_paths):
            distance = all_distances[i]
            feromons_val = feromons_connections_val[i]
            for j in range(len(path)-1):
                #print(self.feromon_matrix.shape)
                #print(self.adjacency_matrix.shape)
                #print(j)
                self.feromon_matrix[path[j]][path[j+1]] += feromons_val                         #self.adjacency_matrix[path[j]][path[j+1]] +

class Algorythm:
    def __init__(self, adjacency_matrix, start) -> None:
        self.adjacency_matrix = adjacency_matrix
        self.start = start

    def tsp_dfs(self):
        all_paths = []
        num_cities = len(self.adjacency_matrix)
        stack = deque([[self.start]])
        while stack:
            path = stack.popleft()
            curr_city = path[-1]
            if len(path) == num_cities:
                all_paths.append(path)
                continue
            for next_city in range(num_cities):
                if next_city not in path and self.adjacency_matrix[curr_city][next_city] != 0:
                    stack.append(path + [next_city])
        return all_paths
        


In [8]:
START_CITY = 0
NUMBER_OF_CITIES = 9

np.set_printoptions(linewidth=np.inf)

variant = int(input("Który wariant? (1, 2, 3, 4)"))
if variant == 1:
    web = WebOfCities(NUMBER_OF_CITIES)
    graph = Graph(web.cities)
    web.create_connections_100()
    adjacency_matrix = graph.create_adjacency_matrix()
elif variant == 2:
    web = WebOfCities(NUMBER_OF_CITIES)
    graph = Graph(web.cities)
    web.create_connections_80()
    adjacency_matrix = graph.create_adjacency_matrix()
elif variant == 3:
    web = WebOfCities(NUMBER_OF_CITIES)
    graph = Graph(web.cities)
    web.create_weighted_connections_100()
    adjacency_matrix = graph.create_adjacency_weighted_matrix()
elif variant == 4:
    web = WebOfCities(NUMBER_OF_CITIES)
    graph = Graph(web.cities)
    web.create_weighted_connections_80()
    adjacency_matrix = graph.create_adjacency_weighted_matrix()
else:
    print("wrong value")
print(adjacency_matrix)
algorythm_ACO = AlgorythmACO(adjacency_matrix=adjacency_matrix)
all_paths = algorythm_ACO.search(2000, 300)

def policz_najczesciej_wystepujaca(lista_list):
    wystapienia = {}
    for lista in lista_list:
        ilosc = lista_list.count(lista)
        wystapienia[str(lista)] = ilosc
    
    najczestsza_lista = max(wystapienia, key=wystapienia.get)
    ilosc_wystapien = wystapienia[najczestsza_lista]
    return najczestsza_lista, ilosc_wystapien
print(policz_najczesciej_wystepujaca(all_paths))
maby_best = policz_najczesciej_wystepujaca(all_paths)[0]
path_count = PathCounter(adjacency_matrix, eval(maby_best))
path_count.count_path()

branch_algorytm = Algorythm(adjacency_matrix, START_CITY)
all_paths = branch_algorytm.tsp_dfs()
path_counter = PathCounter(adjacency_matrix, all_paths)
path_counter.count_all_distances()
path_counter.print_min_possible_path()

[[  0.          49.69909456  50.68530359  46.57252409  70.80960387 100.40916293  93.09135298  46.23851209  25.86503431]
 [ 49.69909456   0.          77.01298592  65.39877675  42.75511665 113.92102528 100.29955134  69.15200648  28.75760769]
 [ 50.68530359  77.01298592   0.          81.4002457   99.7747463   67.50555533 138.45938033  17.23368794  58.13776741]
 [ 46.57252409  65.39877675  81.4002457    0.          98.39207285 142.63590011  60.24118193  84.59905437  60.87692502]
 [ 70.80960387  42.75511665  99.7747463   98.39207285   0.         113.86834503 119.83321743  85.9418408   47.27578661]
 [100.40916293 113.92102528  67.50555533 142.63590011 113.86834503   0.         193.2614809   58.44655678  95.85927185]
 [ 93.09135298 100.29955134 138.45938033  60.24118193 119.83321743 193.2614809    0.         138.22445514 101.79882121]
 [ 46.23851209  69.15200648  17.23368794  84.59905437  85.9418408   58.44655678 138.22445514   0.          48.25971405]
 [ 25.86503431  28.75760769  58.13776741